# <div style="text-align:center; padding:10px; color:white; margin:0; font-size:90%; font-family:'Times New Roman'; background-color:purple; overflow:hidden"><b>               AI Chatbot for Database Interaction     </b></div>                                  <div style="text-align:center; padding:10px; color:white; margin:0; font-size:90%; font-family:'Times New Roman'; background-color:purple; overflow:hidden"><b><b>enabling seamless natural language interaction with any SQL database.</b></div>

## 🧠 Step 1: Connect to Google Gemini LLM
We initialize the **Gemini 2.5 Flash model** from Google Generative AI using the API key stored in the `.env` file.  
This model is responsible for converting **natural language queries** into **SQL commands**.

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.0
)

C:\Users\mpala\anaconda\envs\rag_sql_env\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [12]:
print(llm.invoke("Hello Gemini, are you online?"))

content='Hello! Yes, I am online and ready to help. How can I assist you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--7ce9b202-72fe-4a26-b025-2c4cbf732067-0' usage_metadata={'input_tokens': 8, 'output_tokens': 19, 'total_tokens': 58}


## 🗄️ Step 2: Connect to SQL Server Database
We establish a secure connection with the **Microsoft SQL Server (MSSQL)** database using credentials stored in `.env`.  
This allows the chatbot to access real company data for query generation.

In [15]:
from langchain_community.utilities import SQLDatabase

# --- Database connection parameters ---
db_user = os.getenv("MSSQL_USERNAME")
db_password = os.getenv("MSSQL_PASSWORD")
db_host = os.getenv("MSSQL_SERVER")
db_name = os.getenv("MSSQL_DATABASE")

# --- Create database connection ---
db = SQLDatabase.from_uri(
    f"mssql+pyodbc://{db_user}:{db_password}@{db_host}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server"
)

# --- Test connection ---
print("✅ Connected to database!")

# --- Get list of all tables ---
tables = db.get_usable_table_names()
print("\nAvailable tables:")
print(tables)

# --- Get schema info for all tables ---
info = db.get_table_info()
print("\nFull table schema info:")
print(info)


✅ Connected to database!

Available tables:
['LOCATION', 'department', 'employees']

Full table schema info:

CREATE TABLE [LOCATION] (
	[Location_ID] INTEGER NOT NULL, 
	[City] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__LOCATION__D2BA00C20E80538B] PRIMARY KEY ([Location_ID])
)

/*
3 rows from LOCATION table:
Location_ID	City
122	Salem
123	Dallas
124	Chicago
*/


CREATE TABLE department (
	depart_id INTEGER NOT NULL IDENTITY(1,1), 
	depart_name VARCHAR(30) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__departme__0C1281016E07DF1B] PRIMARY KEY (depart_id)
)

/*
3 rows from department table:
depart_id	depart_name
1	data_scientist
2	data_analyst
3	data_enginer
*/


CREATE TABLE employees (
	employee_id INTEGER NOT NULL, 
	staff_name VARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	department INTEGER NULL, 
	salary MONEY NULL, 
	coundry VARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL DEFAULT ('india'), 
	[Location_ID] INTEGER NULL, 


# <div style="text-align:center; padding:20px; color:white; margin:0; font-size:90%; font-family:'Times New Roman'; background-color:purple; overflow:hidden"><b>***Without Few-Shot Examples***</b></div>

## 🚫 Without Few-Shot Examples
Here, the model directly converts user input to SQL without prior examples.  
This approach may generate simpler or sometimes incomplete SQL statements.

### 🔗 Building a Simple Chain
We use **LangChain’s SQL Query Chain** to link the LLM with our SQL database.  
This lets the model understand schema and generate executable SQL statements automatically.

In [60]:
from langchain.chains import create_sql_query_chain

In [27]:
chain = create_sql_query_chain(llm, db)
def execute_query_1(question):
    try:
        # Generate SQL Query from question
        response = chain.invoke({"question":question})
        print(response)
        print("###################################################")
        #strip the formating markers from the response
        cleaned_query = response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        #execute the cleaned query 
        print("###################################################") 
        result = db.run(cleaned_query)
        print("###################################################")        
        # Display the result
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")

In [25]:
q1 = "show employee name who earn salary more then 5000"
execute_query(q1)

NameError: name 'execute_query_1' is not defined

In [24]:
q2 = "which employees working in the data engeniring department"
execute_query(q2)

NameError: name 'execute_query' is not defined

In [28]:
q3 = "show employers who working in india and they are earn salary more then 5000 and data science department only"
execute_query(q3)

```sql
SELECT TOP 5 [staff_name]
FROM employees
WHERE [coundry] = 'INDIA' AND salary > 5000 AND department = 1;
```
###################################################
SELECT TOP 5 [staff_name]
FROM employees
WHERE [coundry] = 'INDIA' AND salary > 5000 AND department = 1;
###################################################
###################################################
[('suresh',), ('dinesh',)]


# <div style="text-align:center; padding:20px; color:white; margin:0; font-size:90%; font-family:'Times New Roman'; background-color:purple; overflow:hidden"><b>***With Few-Shot Examples***</b></div>

## ✅ With Few-Shot Examples (Improved Accuracy)
Few-shot learning helps the model learn from example question–answer pairs.  
This makes query generation more accurate — especially for complex joins or filters.

In [3]:
few_shots = [
    {'Question': "Show all employee names and their corresponding salaries.",
     'SQLQuery': "SELECT staff_name, salary FROM employees;",
     'SQLResult': "suresh - 20000.00, ajith - 15000.00, arul - 18000.00",
     'Answer': "There are three employees — Suresh earns ₹20,000, Ajith earns ₹15,000, and Arul earns ₹18,000."},

    {'Question': "List all department names.",
     'SQLQuery': "SELECT depart_name FROM department;",
     'SQLResult': "data_scientist, data_analyst, data_enginer",
     'Answer': "The company has three departments — Data Scientist, Data Analyst, and Data Engineer."},

    {'Question': "Show all employees who work in the data scientist department.",
     'SQLQuery': """SELECT e.staff_name, d.depart_name
FROM employees AS e
JOIN department AS d ON e.department = d.depart_id
WHERE d.depart_name = 'data_scientist';""",
     'SQLResult': "suresh - data_scientist",
     'Answer': "Suresh is the only employee working in the Data Scientist department."},

    {'Question': "Which employee has the highest salary?",
     'SQLQuery': """SELECT TOP 1 staff_name, salary
FROM employees
ORDER BY salary DESC;""",
     'SQLResult': "suresh - 20000.00",
     'Answer': "Suresh earns the highest salary — ₹20,000."},

    {'Question': "Which employee earns the lowest salary?",
     'SQLQuery': """SELECT TOP 1 staff_name, salary
FROM employees
ORDER BY salary ASC;""",
     'SQLResult': "ajith - 15000.00",
     'Answer': "Ajith earns the lowest salary — ₹15,000."},

    {'Question': "List all employees along with their department and city.",
     'SQLQuery': """SELECT e.staff_name, d.depart_name, l.City
FROM employees AS e
JOIN department AS d ON e.department = d.depart_id
JOIN LOCATION AS l ON e.Location_ID = l.Location_ID;""",
     'SQLResult': "suresh - data_scientist - Salem, ajith - data_analyst - Dallas, arul - data_enginer - Chicago",
     'Answer': "Suresh works in Salem as a Data Scientist, Ajith in Dallas as a Data Analyst, and Arul in Chicago as a Data Engineer."},

    {'Question': "Show all employees working outside India.",
     'SQLQuery': """SELECT staff_name, coundry
FROM employees
WHERE coundry <> 'INDIA';""",
     'SQLResult': "ajith - USA, arul - CANADA",
     'Answer': "Ajith works in the USA and Arul in Canada — both outside India."},

    {'Question': "Count how many employees are in each department.",
     'SQLQuery': """SELECT d.depart_name, COUNT(e.employee_id) AS employee_count
FROM employees AS e
JOIN department AS d ON e.department = d.depart_id
GROUP BY d.depart_name;""",
     'SQLResult': "data_scientist - 1, data_analyst - 1, data_enginer - 1",
     'Answer': "Each department currently has one employee."},

    {'Question': "Find the average salary of all employees.",
     'SQLQuery': "SELECT AVG(salary) AS average_salary FROM employees;",
     'SQLResult': "17666.67",
     'Answer': "The average employee salary is ₹17,666.67."},

    {'Question': "Show all employees and their department where salary is above ₹15,000.",
     'SQLQuery': """SELECT e.staff_name, e.salary, d.depart_name
FROM employees AS e
JOIN department AS d ON e.department = d.depart_id
WHERE e.salary > 15000;""",
     'SQLResult': "suresh - 20000.00 - data_scientist, arul - 18000.00 - data_enginer",
     'Answer': "Suresh and Arul earn above ₹15,000 — in Data Scientist and Data Engineer departments, respectively."}
]


### 🧩 Step 3: Convert Examples into Vector Embeddings
We convert few-shot examples into numerical **vector embeddings** using a transformer model.  
This helps the LLM find **semantically similar examples** to guide future query generation.

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mssql_prompt
from langchain.chains import create_sql_query_chain

# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Vectorize few-shot examples
to_vectorize = [" ".join(example.values()) for example in few_shots]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

# Semantic similarity selector (retrieves closest few-shot examples dynamically)
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


### 🧾 Step 4: Define Custom SQL Prompt
A structured prompt ensures the LLM follows correct **T-SQL syntax**, avoids mistakes, and uses joins efficiently.  
This prompt acts as the “rule book” for SQL generation.

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mssql_prompt
from langchain.chains import create_sql_query_chain

# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Vectorize few-shot examples
to_vectorize = [" ".join(example.values()) for example in few_shots]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

# Semantic similarity selector (retrieves closest few-shot examples dynamically)
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

mssql_prompt = """
You are an expert SQL developer specialized in Microsoft SQL Server (T-SQL). 
Your task is to translate a natural language question into a syntactically correct and optimal MSSQL query, 
execute it mentally, and then return the logical answer to the input question.

When creating SQL queries:
- Always use correct T-SQL syntax.
- Use only table names and column names that exist in the provided database schema.
- Never invent columns or tables that are not present.
- Prefer explicit JOIN syntax instead of comma joins.
- Use TOP {top_k} only if the question specifically requests a limited number of rows 
  (e.g., “top 5”, “first 10”, etc.).
- Otherwise, do not use TOP or LIMIT.
- Always qualify columns with their table alias when multiple tables are joined.
- Never use backticks (`) — instead, use square brackets [ ] for identifiers.
- Do not include Markdown formatting like ```sql.
- Do not use LIMIT (it’s not supported in SQL Server); use TOP instead if needed.
- Use proper MSSQL date functions like GETDATE() or CAST(GETDATE() AS DATE) when referring to “today”.
- When filtering by text values, always use single quotes ('value').
- Always format keywords (SELECT, FROM, WHERE, GROUP BY, etc.) in uppercase for readability.
- Only select the specific columns required to answer the question, not * (all columns).

After writing the SQL query, logically infer the expected answer from the query results.

Use the following format strictly:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble, no commentary — only follow the exact format above.
"""



# STEP 3: Create FewShot Prompt Template

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mssql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],  # These variables are required
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


## 🧮 Step 5: Text-to-SQL Query Execution
Here, the user enters a **natural-language question**, which the LLM converts into an optimized SQL query.  
The query is executed, and results are retrieved from the connected database.

In [22]:
rag_chain = create_sql_query_chain(llm, db, prompt=few_shot_prompt)
def execute_query(question):
    try:
        # 🧠 Step 1: Generate SQL Query from LLM
        response = rag_chain.invoke({"question": question})
        print(f"Question:\n{question}")
        print("###################################################\n")
        print(f"MSSQL_QUERY:\n{response}")
        print("###################################################\n")

        # 🧹 Step 2: Clean query formatting
        cleaned_query = response.strip('```sql\n').strip('\n```')
        print(f"Cleaned Query:\n{cleaned_query}\n")

        # 🧮 Step 3: Execute the cleaned query 
        result = db.run(cleaned_query)
        print("###################################################\n")
        print(f"Query Result:\n{result}\n")

        # 🗣️ Step 4: Ask LLM to interpret + generate equivalent queries
        llm_input = f"""
        You are the most experienced SQL and data analytics expert.
        You are given a user's question, the corresponding MSSQL query, and its result.
        You must:
        1️⃣ Explain the answer briefly in 1–3 sentences.
        2️⃣ Provide equivalent SQL queries for the same logic in MySQL and Oracle syntax.
        3️⃣ Display the result as a professional table comparison in this format:

        ┌────────────┬──────────────────────────────────────────────┐
        │ Database   │ Equivalent SQL Query                         │
        ├────────────┼──────────────────────────────────────────────┤
        │ MSSQL      │ <MSSQL version>                              │
        │ MySQL      │ <MySQL version>                              │
        │ Oracle     │ <Oracle version>                             │
        └────────────┴──────────────────────────────────────────────┘

        Question: {question}
        SQLQuery (MSSQL): {cleaned_query}
        SQLResult: {result}

        Answer:
        """
        llm_answer = llm.invoke(llm_input)
        print("###################################################\n")
        print("LLM Answer (Interpretation):")
        print(llm_answer.content)

    except ProgrammingError as e:
        print(f"An error occurred: {e}")

## 💬 Step 6: LLM Interpretation
Finally, the chatbot interprets the SQL result in plain English and generates **equivalent SQL queries**  
for **MySQL** and **Oracle** — showcasing multi-database adaptability.

In [54]:
        # 🗣️ Step 4: Ask LLM to interpret + generate equivalent queries
        llm_input = f"""
        You are the most experienced SQL and data analytics expert.
        You are given a user's question, the corresponding MSSQL query, and its result.
        You must:
        1️⃣ Explain the answer briefly in 1–3 sentences.
        2️⃣ Provide equivalent SQL queries for the same logic in MySQL and Oracle syntax.
        3️⃣ Display the result as a professional table comparison in this format:

        ┌────────────┬──────────────────────────────────────────────┐
        │ Database   │ Equivalent SQL Query                         │
        ├────────────┼──────────────────────────────────────────────┤
        │ MSSQL      │ <MSSQL version>                              │
        │ MySQL      │ <MySQL version>                              │
        │ Oracle     │ <Oracle version>                             │
        └────────────┴──────────────────────────────────────────────┘

        Question: {question}
        SQLQuery (MSSQL): {cleaned_query}
        SQLResult: {result}

        Answer:
        """
        llm_answer = llm.invoke(llm_input)
        print("###################################################\n")
        print("LLM Answer (Interpretation):")
        print(llm_answer.content)

    except ProgrammingError as e:
        print(f"An error occurred: {e}")

IndentationError: unexpected indent (2996204191.py, line 29)

In [24]:
execute_query("show employers who working in india and they are earn salary more then 5000 and data science department only")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Question:
show employers who working in india and they are earn salary more then 5000 and data science department only
###################################################

MSSQL_QUERY:
SELECT e.staff_name, e.salary
FROM employees AS e
JOIN department AS d ON e.department = d.depart_id
WHERE e.coundry = 'INDIA'
  AND e.salary > 5000
  AND d.depart_name = 'data_scientist';
###################################################

Cleaned Query:
SELECT e.staff_name, e.salary
FROM employees AS e
JOIN department AS d ON e.department = d.depart_id
WHERE e.coundry = 'INDIA'
  AND e.salary > 5000
  AND d.depart_name = 'data_scientist';

###################################################

Query Result:
[('suresh', Decimal('20000.0000')), ('dinesh', Decimal('12000.0000'))]

###################################################

LLM Answer (Interpretation):
The query identifies employees who work in India, earn more than 5000, and are part of the 'data_scientist' department. The result shows 'suresh' a